In [9]:
from Data_visualization.Feature_selection import FeatImp
from Data_cleanup.Cleanup import BlueRedSubstraction,Normalisation
from data_collect import detect_outliers
from Training.Training import cross_val_results
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


data = pd.read_csv('data/lol.csv') 
data_0 = data.drop(['gameId'],axis = 1)
target = data['blueWins']
data_1 = BlueRedSubstraction(data) #spremenjeni podatki
data_2 = pd.concat([data_1,target],axis = 1)
data_3 = data_2.drop(["GoldPerMinDiff","blueWins","CSPerMinDiff"],axis = 1) # ovržem nekatere podobne stolpce,ter target stolpec
train_len = int(data_3.shape[0]*0.8)
train = data_3[:train_len]
test = data_3[train_len:]

target_train = target[:train_len]
target_test = target[train_len:]
norm = Normalisation(MM = True)
train_sc, train_mm = norm.transform(X = train)

In [10]:
'''Fine tuning Random forest Classifier'''
RFC = RandomForestClassifier()
kfold = KFold(n_splits=10)

## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[10,100,1000,50],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= -1, verbose = 1)

gsRFC.fit(train_sc,target_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  4.5min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 10.6min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 29.7min
[Parallel(n_jobs=4)]: Done 1080 out of 1080 | elapsed: 54.9min finished


0.7295944886299989

In [15]:
print(gsRFC.best_params_,)

{'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 1, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 1000} {'mean_fit_time': array([  0.21483839,   1.90071349,  23.14857376,   0.97016082,
         0.19686854,   1.81453462,  19.02204916,   0.93447933,
         0.17361882,   1.72550738,  16.21692045,   0.79009039,
         0.16527472,   1.53954546,  15.99199908,   0.76231425,
         0.15737674,   1.53555398,  15.50589771,   0.78186464,
         0.17402465,   1.53757741,  15.87934003,   0.77747397,
         0.16062846,   1.33361647,  12.49316809,   0.58662062,
         0.14072003,   1.14613338,  12.19475858,   0.58636696,
         0.12728825,   1.12760782,  11.70217881,   0.57666459,
         0.39283748,   3.53887069,  37.54333687,   2.24874957,
         0.43822811,   3.40465577,  33.19904556,   1.65727167,
         0.31164765,   2.99002249,  32.85490255,   1.5598609 ,
         0.33330002,   3.41800516,  34.52893069,   1.69154243,
         0.31